<a href="https://colab.research.google.com/github/osung/openai/blob/main/summarize_llm_lg_exa_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize

In [2]:
from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
import time

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/colab_run/summary_output.pkl')
display(df.head())

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."


In [ ]:
df_max_scores = pd.read_pickle('/content/drive/MyDrive/colab_run/df_max_scores_exaone.pkl')

In [ ]:
df_ksic_small = pd.read_excel('/content/drive/MyDrive/colab_run/ksic_small.xlsx')

In [ ]:
df_ksic_small.rename(columns={'코드': 'ksic', '항목명': '산업명'}, inplace=True)

In [ ]:
df_news_ksic = pd.merge(df, df_ksic_small, on='ksic', how='left')

In [ ]:
df_news_ksic

,id,topic,text,ksic,max_scores,summary_new,산업명
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,기타 식품 제조업
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,수산물 가공 및 저장 처리업
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,"과실, 채소 가공 및 저장 처리업"
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,"과실, 채소 가공 및 저장 처리업"
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...","과실, 채소 가공 및 저장 처리업"
...,...,...,...,...,...,...,...
765021,NWRW1900000038.41782,T,네이버·다음 실시간 검색어 순위 ‘정부 요청 시 삭제’ 규정 운영했다 ㆍ자동완성·연...,NaN,0.000000,국내 양대 포털사이트인 네이버와 카카오(다음)가 정부 당국이 요청할 경우 실시간 급...,NaN
765022,NLRW2000000006.29284,T,"[미래사회포럼]최정준 동방문화대학원대 교수, ""소명 알고 완성하는게 천명 이루는 길...",NaN,0.000000,22일 열린 미래사회포럼 강연자로 나선 최정준 동방문화대학원대학교 미래예측학과 교수...,NaN
765023,NLRW2000000006.29247,T,"원더브라88원, 8월 8일 기념해 초대형 프로모션 '행사 가격은?' 라이프스타일 전...",NaN,0.000000,라이프스타일 전문기업 그리티가 푸쉬업 브라브랜드 '원더브라' 브랜드데이를 맞아 8일...,NaN
765024,NWRW1900000038.41262,T,마이클 코스털리츠 방한 “남다른 시각과 국가적 투자가 노벨상 만들어” ㆍ올해 노벨물...,NaN,0.000000,올해 노벨 물리학상 수상자 중 한 명인 마이클 코스털리츠 미국 브라운대 교수는 20...,NaN


In [ ]:
device = torch.device("cuda")
model_name = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [ ]:
model = model.half()
model = model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
max_token_length = getattr(model.config, "max_position_embeddings", 512)

In [ ]:
max_token_length

32768

In [ ]:
get_new_tokens = 256
prompt_max = max(16, max_token_length - get_new_tokens)

In [ ]:
prompt_max

32512

In [ ]:
df_worst_scores = df[df['max_scores'] < 0.3]

In [ ]:
def after_assistant(text: str) -> str:
    _, sep, tail = text.partition("[|assistant|]")
    return tail if sep else ""   # 구분자가 없으면 빈 문자열

In [ ]:
def get_summarize(text, ksic='어떤 산업') :

  #role = "너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 어떤 산업에 대한 뉴스 기사인데, 이것을 요약해줘. \
  #        존대말을 사용하지 말고, 문장을 ~다. 로 서술형으로 끝내도록 해줘."

  role = f"너는 뉴스를 요약하는 요약 전문가야. 다음 뉴스 기사를 {ksic}의 관점에서 요약해줘. 반드시 {ksic}과 관련된 핵심 단어를 포함해서 요약문을 작성해줘.\
           존대말을 사용하지 말고, 문장을 ~다. 로 서술형으로 끝내도록 해줘. \
           절대로 핵심 단어를 별도로 정리해서 나열하거나 다른 어떤 관점에서 작성했는지 설명없이 그냥 기사만 요약하고 끝내줘."


  messages = [
    {"role": "system",
     "content": role},
    {"role": "user", "content": text}
  ]

  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
  )

  output = model.generate(
    inputs.to("cuda"),
    #eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=max_token_length,
    do_sample=False,
    stop_strings=["<|endofturn|>", "<|stop|>"],
    tokenizer=tokenizer
  )

  responses = tokenizer.decode(output[0], skip_special_tokens=True)

  return after_assistant(responses)

In [ ]:
text = df_worst_scores.loc[4361]['text']
text

'바나나 전염병에 스미후루코리아 “국민과일 바나나의 매력!" 밝혀... 바나나 전염병에 스미후루코리아 “국민과일 바나나의 매력!" 밝혀... 바나나 전염병이 공개되면서 바나나가 멸종위기에 처해있다. 이러한 가운데 스미후루코리아 측은 “국민과일 바나나의 매력!”이라며 보도자료를 보냈다. 바나나 전염병에 맞서는 스미후루 코리아 측은 \'국민 과일\'은 무엇일까. 바나나다. 한 대형마트의 최근 조사에 따르면 바나나가 3년 연속 ‘가장 많이 팔리는 과일 1위’에 올랐다. 수박을 비롯해 감귤-사과-딸기 등 쟁쟁한 과일들을 앞선 결과다. 글로벌 청과 기업 \'스미후루코리아(www.sumifru.co.kr)\'는 \'국민과일\'로 등극한 바나나의 매력과 이유를 분석해 소개했다. 사시사철 저렴한 가격에 구입할 수 있다는 점은 바나나가 대중적으로 사랑 받는 장점. 이와 함께 바나나는 특유의 부드러운 과육과 매력적인 향으로 식욕을 돋군다. 이 때문에 아이들의 이유식으로, 성인 다이어트 식품으로 각광받고 있다. 바나나의 진정한 매력은 저렴한 가격뿐 아니라 풍부한 영양소에 있다. 또 다른 ‘국민과일’로 꼽히는 \'사과\'와 비교했을 때, 바나나의 영양이 얼마나 훌륭한 지 알 수 있다. 사과에는 100g 기준으로 칼로리가 75kcal 함유돼 있다. 반면, 바나나는 93kcal로 열량이 더 높다. 운동할 때 바나나를 섭취하면 많은 에너지를 빠르게 섭취할 수 있다. 바나나가 사과보다 칼로리가 높다고 해서 많이 섭취하면 다이어트에 방해가 되지 않을 지 걱정하는 경우가 있는데, 꼭 그렇지만은 않다. 바나나는 식이섬유가 다른 과일에 비해 풍부하다. 그래서 포만감을 쉽게 일으켜 폭식을 억제한다. 바나나에 다량 함유된 식이섬유는 고분자 탄수화물이다. 이 성분은 콜레스테롤 흡수를 막는다. 그 덕분에 성인병 예방에 탁월한 효과가 있으며, 위장의 공복감을 덜 느끼게 한다. 게다가 바나나에는 풍부한 칼륨(100g 당 358mg)이 함유돼 있다. 이는 사과(107mg)보다 3배 이상 높은 수치다. 바나나

In [ ]:
get_summarize(text, '작물 재배업')

'바나나 전염병이 확산되며 바나나의 미래에 대한 우려가 커지고 있지만, 스미후루코리아는 바나나의 지속적인 인기와 가치를 강조했다. 바나나는 3년 연속 가장 많이 팔리는 과일로 자리매김하며, 저렴한 가격과 풍부한 영양소로 대중의 사랑을 받고 있다. 특히, 바나나는 높은 칼로리와 풍부한 식이섬유, 칼륨 함량으로 인해 에너지 공급원 및 건강식품으로 주목받고 있다. 스미후루코리아는 바나나의 이러한 장점을 부각하며, 전염병에도 불구하고 바나나의 영양적 가치와 대중적 매력을 재확인했다. 국내에서는 연간 수백만 상자가 수입되어 다양한 연령대의 식단에 필수적인 작물로 자리잡고 있다. 그럼에도 불구하고 전염병의 위협은 바나나 재배업에 심각한 도전을 제기하고 있다.'

In [ ]:
get_summarize(text, '의약품 제조업')

'바나나 전염병이 확산되며 바나나의 미래에 대한 우려가 커지고 있지만, 스미후루코리아는 바나나의 지속적인 인기와 가치를 강조했다. 바나나는 저렴한 가격과 풍부한 영양소로 인해 국민과일로 자리매김했다. 특히, 바나나는 높은 칼로리와 풍부한 식이섬유로 에너지 공급과 포만감을 동시에 제공하며, 칼륨 함량이 높아 혈압 조절과 붓기 제거에 효과적이다. 이러한 특성 덕분에 아이들의 이유식부터 성인의 다이어트 식품까지 폭넓게 사랑받고 있다. 스미후루코리아는 글로벌 네트워크를 바탕으로 바나나를 포함한 열대과일의 안정적인 공급을 위해 노력하고 있으며, 국내로 연간 수백만 상자를 수입하여 유통하고 있다. 전염병 위기 속에서도 바나나의 영양적 이점과 대중적 인기는 여전히 견고함을 보여주고 있다.'

In [ ]:
df

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."
...,...,...,...,...,...,...
765021,NWRW1900000038.41782,T,네이버·다음 실시간 검색어 순위 ‘정부 요청 시 삭제’ 규정 운영했다 ㆍ자동완성·연...,NaN,0.000000,국내 양대 포털사이트인 네이버와 카카오(다음)가 정부 당국이 요청할 경우 실시간 급...
765022,NLRW2000000006.29284,T,"[미래사회포럼]최정준 동방문화대학원대 교수, ""소명 알고 완성하는게 천명 이루는 길...",NaN,0.000000,22일 열린 미래사회포럼 강연자로 나선 최정준 동방문화대학원대학교 미래예측학과 교수...
765023,NLRW2000000006.29247,T,"원더브라88원, 8월 8일 기념해 초대형 프로모션 '행사 가격은?' 라이프스타일 전...",NaN,0.000000,라이프스타일 전문기업 그리티가 푸쉬업 브라브랜드 '원더브라' 브랜드데이를 맞아 8일...
765024,NWRW1900000038.41262,T,마이클 코스털리츠 방한 “남다른 시각과 국가적 투자가 노벨상 만들어” ㆍ올해 노벨물...,NaN,0.000000,올해 노벨 물리학상 수상자 중 한 명인 마이클 코스털리츠 미국 브라운대 교수는 20...


In [ ]:
df_max_scores = df[df['max_scores'] > 0.38]

In [ ]:
df_max_scores

,id,topic,text,ksic,max_scores,summary_new,summary_exaone
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이인 1221...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,고령 우곡 지역의 수박 농가들이 올해 이상기후로 인해 수확량이 급감하고 수박 품질이...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...",국내 친환경 농산물 생산량이 전체 농산물 생산량의 11.9%를 넘어서며 10%를 처...
...,...,...,...,...,...,...,...
382785,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 판매로 큰...
382786,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...,삼성전자의 갤럭시S9과 갤럭시S9+가 유럽 6개국의 소비자 평가에서 애플의 아이폰 ...
382787,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...,"삼성전자가 스위스 국영 철도청(SBB)에 갤럭시노트3, 갤럭시S4, 갤럭시탭3 등을..."
382788,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 최고 스마...


In [ ]:
len(df_max_scores['ksic'].value_counts())

41

In [ ]:
df_max_scores['summary_exaone'] = df_max_scores['text'].progress_apply(get_summarize)

  0%|          | 0/984 [00:00<?, ?it/s]

/tmp/ipython-input-405031340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_scores['summary_exaone'] = df_max_scores['text'].progress_apply(get_summarize)


In [ ]:
df_max_scores.to_pickle('df_max_scores_exaone.pkl')

In [ ]:
df_max_scores_ksic = pd.merge(df_max_scores, df_ksic_small, on='ksic', how='left')

In [ ]:
df_max_scores_ksic

,id,topic,text,ksic,max_scores,summary_new,summary_exaone,산업명
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이인 1221...,기타 식품 제조업
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...,수산물 가공 및 저장 처리업
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,고령 우곡 지역의 수박 농가들이 올해 이상기후로 인해 수확량이 급감하고 수박 품질이...,"과실, 채소 가공 및 저장 처리업"
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농...,"과실, 채소 가공 및 저장 처리업"
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...",국내 친환경 농산물 생산량이 전체 농산물 생산량의 11.9%를 넘어서며 10%를 처...,"과실, 채소 가공 및 저장 처리업"
...,...,...,...,...,...,...,...,...
979,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 판매로 큰...,통신 및 방송장비 제조업
980,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...,삼성전자의 갤럭시S9과 갤럭시S9+가 유럽 6개국의 소비자 평가에서 애플의 아이폰 ...,통신 및 방송장비 제조업
981,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...,"삼성전자가 스위스 국영 철도청(SBB)에 갤럭시노트3, 갤럭시S4, 갤럭시탭3 등을...",통신 및 방송장비 제조업
982,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 최고 스마...,통신 및 방송장비 제조업


In [ ]:
get_summarize(df_max_scores_ksic.iloc[0]['text'], df_max_scores_ksic.iloc[0]['산업명'])

'스위스 제빵사협회가 창립 25주년을 기념해 길이 1221m의 거대한 초콜릿 파이를 제작해 주목받았다. 이 파이는 크림, 우유, 밀가루, 버터, 초콜릿 등 대량의 재료를 사용해 총 무게 4천207kg에 달하며, 약 3만 명의 소비자를 위한 규모다. 제조 과정에는 전문 제빵사 25명과 일반인 175명이 참여해 3시간 이상 소요되었고, 이 파이는 ‘세계에서 가장 긴 밀푀유 파이’로 기네스북에 등재되며 기존 기록을 경신했다. 식품 제조업 관점에서 보면, 이 사건은 대량 생산 기술과 창의적인 제품 개발의 가능성을 보여주며, 특히 파이 제조 분야에서의 혁신적 접근법과 마케팅 효과를 강조한다.'

In [ ]:
df_max_scores_ksic['summary_exaone_ksic'] = df_max_scores_ksic['text'].progress_apply(get_summarize, ksic=df_max_scores_ksic['산업명'])

  0%|          | 0/984 [00:00<?, ?it/s]

In [ ]:
df_max_scores_ksic.to_pickle('/content/drive/MyDrive/colab_run/df_max_scores_exaone_ksic.pkl')

In [ ]:
df_max_scores_ksic = pd.read_pickle('/content/drive/MyDrive/colab_run/df_max_scores_exaone_ksic.pkl')

In [ ]:
df_max_scores_ksic

,id,topic,text,ksic,max_scores,summary_new,summary_exaone,산업명,summary_exaone_ksic
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이인 1221...,기타 식품 제조업,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이를 제작했다...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...,수산물 가공 및 저장 처리업,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,고령 우곡 지역의 수박 농가들이 올해 이상기후로 인해 수확량이 급감하고 수박 품질이...,"과실, 채소 가공 및 저장 처리업",고령 우곡 지역의 고령 농민들은 올해 이상기후로 인해 수박 농사가 큰 타격을 입었다...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농...,"과실, 채소 가공 및 저장 처리업",태안군에서 해풍을 맞고 자란 명품 양파 수확이 한창이다. 약 150여 농가가 76h...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...",국내 친환경 농산물 생산량이 전체 농산물 생산량의 11.9%를 넘어서며 10%를 처...,"과실, 채소 가공 및 저장 처리업",국내 친환경 농산물 생산량이 급증하여 전체 농산물 생산량의 11.9%를 넘어섰다. ...
...,...,...,...,...,...,...,...,...,...
979,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 판매로 큰...,통신 및 방송장비 제조업,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 공급으로 ...
980,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...,삼성전자의 갤럭시S9과 갤럭시S9+가 유럽 6개국의 소비자 평가에서 애플의 아이폰 ...,통신 및 방송장비 제조업,"삼성전자의 갤럭시S9 시리즈가 유럽 6개국(이탈리아, 프랑스, 포르투갈, 스페인, ..."
981,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...,"삼성전자가 스위스 국영 철도청(SBB)에 갤럭시노트3, 갤럭시S4, 갤럭시탭3 등을...",통신 및 방송장비 제조업,"삼성전자가 스위스 국영 철도청(SBB)의 모바일 오피스 전용 단말기로 갤럭시노트3,..."
982,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 최고 스마...,통신 및 방송장비 제조업,삼성전자의 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 열린 모바일 월드 콩그...


In [ ]:
device = torch.device("cuda")
model_name = 'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

config.json: 0.00B [00:00, ?B/s]

configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

In [ ]:
df_max_scores_ksic['summary_exaone_mini_ksic'] = df_max_scores_ksic['text'].progress_apply(get_summarize, ksic=df_max_scores_ksic['산업명'])

  0%|          | 0/984 [00:00<?, ?it/s]

In [ ]:
df_max_scores_ksic

,id,topic,text,ksic,max_scores,summary_new,summary_exaone,산업명,summary_exaone_ksic,summary_exaone_mini_ksic
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이인 1221...,기타 식품 제조업,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이를 제작했다...,스위스의 제빵사협회가 창립 기념으로 세계 기록에 도전하는 거대한 규모의 초콜릿 파이...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...,수산물 가공 및 저장 처리업,고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...,최근 20년간 연근해 수산물 중 어획량이 크게 감소한 어종 중 하나로 고등어가 주목...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...,고령 우곡 지역의 수박 농가들이 올해 이상기후로 인해 수확량이 급감하고 수박 품질이...,"과실, 채소 가공 및 저장 처리업",고령 우곡 지역의 고령 농민들은 올해 이상기후로 인해 수박 농사가 큰 타격을 입었다...,고령 지역의 수박농민들이 올해 농사로 인해 큰 어려움을 겪고 있다는 소식이 전해졌다...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...,태안군에서 해풍을 맞고 자란 명품 양파 수확이 활발히 진행 중이다. 약 150개 농...,"과실, 채소 가공 및 저장 처리업",태안군에서 해풍을 맞고 자란 명품 양파 수확이 한창이다. 약 150여 농가가 76h...,태안군에서는 해풍을 맞고 자란 명품 양파가 본격적으로 수확되고 있어 주목받고 있다....
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,...",국내 친환경 농산물 생산량이 전체 농산물 생산량의 11.9%를 넘어서며 10%를 처...,"과실, 채소 가공 및 저장 처리업",국내 친환경 농산물 생산량이 급증하여 전체 농산물 생산량의 11.9%를 넘어섰다. ...,친환경 농산물의 시장 점유율이 크게 상승하여 국내 농산물 생산량의 약 10%를 넘어...
...,...,...,...,...,...,...,...,...,...,...
979,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 판매로 큰...,통신 및 방송장비 제조업,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 공급으로 ...,삼성전자는 애플의 아이폰 성공으로 인해 상당한 경제적 이익을 얻고 있다. 아이폰의 ...
980,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...,삼성전자의 갤럭시S9과 갤럭시S9+가 유럽 6개국의 소비자 평가에서 애플의 아이폰 ...,통신 및 방송장비 제조업,"삼성전자의 갤럭시S9 시리즈가 유럽 6개국(이탈리아, 프랑스, 포르투갈, 스페인, ...",삼성전자의 갤럭시S9 시리즈가 유럽 소비자 평가에서 애플의 아이폰을 압도하며 1위와...
981,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...,"삼성전자가 스위스 국영 철도청(SBB)에 갤럭시노트3, 갤럭시S4, 갤럭시탭3 등을...",통신 및 방송장비 제조업,"삼성전자가 스위스 국영 철도청(SBB)의 모바일 오피스 전용 단말기로 갤럭시노트3,...","삼성전자가 갤럭시노트3와 갤럭시S4, 갤럭시탭3를 스위스 국영 철도청(SBB)의 모..."
982,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 최고 스마...,통신 및 방송장비 제조업,삼성전자의 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 열린 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 '최고의 ...


In [ ]:
df_max_scores_ksic.to_pickle('/content/drive/MyDrive/colab_run/df_max_scores_exaone_mini_ksic.pkl')

In [6]:
df_max_scores_ksic = pd.read_pickle('/content/drive/MyDrive/colab_run/df_max_scores_exaone_mini_ksic.pkl')

In [9]:
df_max_scores_ksic[['text', 'summary_exaone_ksic', 'summary_exaone_mini_ksic']]

,text,summary_exaone_ksic,summary_exaone_mini_ksic
0,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,스위스 제빵사협회가 창립 25주년을 기념해 세계에서 가장 긴 초콜릿 파이를 제작했다...,스위스의 제빵사협회가 창립 기념으로 세계 기록에 도전하는 거대한 규모의 초콜릿 파이...
1,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",고등어 어획량이 20년 동안 3분의 1로 급감했다. 1996년 41만5000톤에서 ...,최근 20년간 연근해 수산물 중 어획량이 크게 감소한 어종 중 하나로 고등어가 주목...
2,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",고령 우곡 지역의 고령 농민들은 올해 이상기후로 인해 수박 농사가 큰 타격을 입었다...,고령 지역의 수박농민들이 올해 농사로 인해 큰 어려움을 겪고 있다는 소식이 전해졌다...
3,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",태안군에서 해풍을 맞고 자란 명품 양파 수확이 한창이다. 약 150여 농가가 76h...,태안군에서는 해풍을 맞고 자란 명품 양파가 본격적으로 수확되고 있어 주목받고 있다....
4,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,국내 친환경 농산물 생산량이 급증하여 전체 농산물 생산량의 11.9%를 넘어섰다. ...,친환경 농산물의 시장 점유율이 크게 상승하여 국내 농산물 생산량의 약 10%를 넘어...
...,...,...,...
979,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,삼성전자는 애플의 아이폰 덕분에 CPU와 낸드플래시 메모리 등 핵심 부품 공급으로 ...,삼성전자는 애플의 아이폰 성공으로 인해 상당한 경제적 이익을 얻고 있다. 아이폰의 ...
980,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...","삼성전자의 갤럭시S9 시리즈가 유럽 6개국(이탈리아, 프랑스, 포르투갈, 스페인, ...",삼성전자의 갤럭시S9 시리즈가 유럽 소비자 평가에서 애플의 아이폰을 압도하며 1위와...
981,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,"삼성전자가 스위스 국영 철도청(SBB)의 모바일 오피스 전용 단말기로 갤럭시노트3,...","삼성전자가 갤럭시노트3와 갤럭시S4, 갤럭시탭3를 스위스 국영 철도청(SBB)의 모..."
982,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",삼성전자의 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 열린 모바일 월드 콩그...,삼성전자의 갤럭시 S21 울트라 5G가 모바일 월드 콩그레스 2021에서 '최고의 ...
